In [1]:
from svpg.utils import load_algo
from svpg.algos.svpg import RBF

import torch
import torch.nn as nn
from torch.nn.utils import parameters_to_vector

from pathlib import Path

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [17]:
action_agents, _, _, _ = load_algo("../runs/RllCartPoleSwingUp-v0/22-05-13/17-08-30/A2C-independent")

theta_i = action_agents[0].parameters()
theta_j = action_agents[1].parameters()

params = [
    parameters_to_vector(action_agent.parameters())
    for action_agent in action_agents
]
params = torch.stack(params)

kernel = RBF()(params, params.detach())

In [41]:
directory = "../runs/RllCartPoleSwingUp-v0/22-05-13/17-08-30/"
(Path(directory).parent.name + Path(directory).name).replace('-', '')

'220513170830'